# Tests for spatial filtering for NATL60

__purpose__ : trying to (!) optimize filtering for NATL60 using xarray and dask

In [1]:
## magics

%load_ext version_information 
%version_information numpy,xarray, dask

Software versions
Python 2.7.11 64bit [GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
IPython 4.2.0
OS Linux 3.16.0 4 amd64 x86_64 with debian 8.5
numpy 1.11.1
xarray 0.9.1
dask 0.10.0
Fri Mar 10 13:49:08 2017 CET

In [2]:
## path for mdules

import sys
sys.path.insert(0,"/home/albert/lib/python")

In [3]:
## imports

import numpy as np
import dask
import xarray as xr
import GriddedData
import time

#- Other modules
import os
from glob import glob
import numpy.ma as ma
import scipy as sc
import WavenumberSpectrum as ws
import dask.array as da

### palette
from matplotlib.colors import LogNorm
import matplotlib.cm as mplcm
import colormap as cmaps
import matplotlib.cm as cm

import seaborn as sns

seq_cmap = mplcm.Blues
div_cmap = mplcm.seismic

### quick plot
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

## local/specific imports
import oocgcm
import oocgcm.filtering
import oocgcm.filtering.linearfilters as tf
%matplotlib inline

In [4]:
## Data

dir="/media/extra/DATA/NATL60/NATL60-CJM165-S/"
files03="NATL60-CJM165_y2013m03d*.1d_curloverf.nc"
files09="NATL60-CJM165_y2013m09d*.1d_curloverf.nc"

tfiles03=dir+files03
tfiles09=dir+files09


In [5]:
curls = xr.open_mfdataset(tfiles03,concat_dim='time_counter')['socurloverf'].chunk(chunks={'x': 1000, 'y':1000, 'time_counter': 1})

In [6]:
curl0=curls[0]

In [7]:
win_box2D = curl0.win

In [8]:
%timeit win_box2D.set(window_name='lanczos', n=[120, 120], dims=['x', 'y'], fc=0.00833)

/home/albert/lib/python/oocgcm/filtering/linearfilters.py:58: RuntimeWarning: invalid value encountered in divide
  np.sin(np.pi * k / (n / 2.)) / (np.pi * k / (n / 2.)))


The slowest run took 8.59 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 745 µs per loop


In [28]:
bw = win_box2D.boundary_weights(drop_dims=[])


In [29]:
signal_LS = win_box2D.apply(weights=bw)

[########                                ] | 20% Completed | 45.7s


MemoryError: 

Traceback
---------
  File "/home/albert/anaconda2/lib/python2.7/site-packages/dask/async.py", line 267, in execute_task
    result = _execute_task(task, data)
  File "/home/albert/anaconda2/lib/python2.7/site-packages/dask/async.py", line 248, in _execute_task
    args2 = [_execute_task(a, cache) for a in args]
  File "/home/albert/anaconda2/lib/python2.7/site-packages/dask/async.py", line 249, in _execute_task
    return func(*args2)
  File "/home/albert/lib/python/oocgcm/filtering/linearfilters.py", line 202, in convolve
    xf = im.convolve(x, self.coefficients, mode=mode)
  File "/home/albert/anaconda2/lib/python2.7/site-packages/scipy/ndimage/filters.py", line 755, in convolve
    origin, True)
  File "/home/albert/anaconda2/lib/python2.7/site-packages/scipy/ndimage/filters.py", line 603, in _correlate_or_convolve
    _nd_image.correlate(input, weights, output, mode, cval, origins)


In [30]:
curls = xr.open_mfdataset(tfiles03,concat_dim='time_counter')['socurloverf'].chunk(chunks={'x': 500, 'y':500, 'time_counter': 1})
curl0=curls[0]
win_box2D = curl0.win
win_box2D.set(window_name='lanczos', n=[120, 120], dims=['x', 'y'], fc=0.00833)
%timeit bw = win_box2D.boundary_weights(drop_dims=[])
signal_LS = win_box2D.apply(weights=bw)

1 loop, best of 3: 17min 11s per loop
[##                                      ] | 6% Completed | 45.5s


MemoryError: 

Traceback
---------
  File "/home/albert/anaconda2/lib/python2.7/site-packages/dask/async.py", line 267, in execute_task
    result = _execute_task(task, data)
  File "/home/albert/anaconda2/lib/python2.7/site-packages/dask/async.py", line 248, in _execute_task
    args2 = [_execute_task(a, cache) for a in args]
  File "/home/albert/anaconda2/lib/python2.7/site-packages/dask/async.py", line 249, in _execute_task
    return func(*args2)
  File "/home/albert/lib/python/oocgcm/filtering/linearfilters.py", line 202, in convolve
    xf = im.convolve(x, self.coefficients, mode=mode)
  File "/home/albert/anaconda2/lib/python2.7/site-packages/scipy/ndimage/filters.py", line 755, in convolve
    origin, True)
  File "/home/albert/anaconda2/lib/python2.7/site-packages/scipy/ndimage/filters.py", line 603, in _correlate_or_convolve
    _nd_image.correlate(input, weights, output, mode, cval, origins)


In [31]:
curls = xr.open_mfdataset(tfiles03,concat_dim='time_counter')['socurloverf'].chunk(chunks={'x': 1500, 'y':1500, 'time_counter': 1})
curl0=curls[0]
win_box2D = curl0.win
win_box2D.set(window_name='lanczos', n=[120, 120], dims=['x', 'y'], fc=0.00833)
%timeit bw = win_box2D.boundary_weights(drop_dims=[])
signal_LS = win_box2D.apply(weights=bw)

1 loop, best of 3: 16min 47s per loop
[###############                         ] | 39% Completed | 45.4s


MemoryError: 

Traceback
---------
  File "/home/albert/anaconda2/lib/python2.7/site-packages/dask/async.py", line 267, in execute_task
    result = _execute_task(task, data)
  File "/home/albert/anaconda2/lib/python2.7/site-packages/dask/async.py", line 248, in _execute_task
    args2 = [_execute_task(a, cache) for a in args]
  File "/home/albert/anaconda2/lib/python2.7/site-packages/dask/async.py", line 249, in _execute_task
    return func(*args2)
  File "/home/albert/lib/python/oocgcm/filtering/linearfilters.py", line 202, in convolve
    xf = im.convolve(x, self.coefficients, mode=mode)
  File "/home/albert/anaconda2/lib/python2.7/site-packages/scipy/ndimage/filters.py", line 755, in convolve
    origin, True)
  File "/home/albert/anaconda2/lib/python2.7/site-packages/scipy/ndimage/filters.py", line 603, in _correlate_or_convolve
    _nd_image.correlate(input, weights, output, mode, cval, origins)


In [32]:
curls = xr.open_mfdataset(tfiles03,concat_dim='time_counter')['socurloverf'].chunk(chunks={'x': 2000, 'y':2000, 'time_counter': 1})
curl0=curls[0]
win_box2D = curl0.win
win_box2D.set(window_name='lanczos', n=[120, 120], dims=['x', 'y'], fc=0.00833)
%timeit bw = win_box2D.boundary_weights(drop_dims=[])
signal_LS = win_box2D.apply(weights=bw)

1 loop, best of 3: 16min 36s per loop
[#####################################   ] | 93% Completed | 46.3s


MemoryError: 

Traceback
---------
  File "/home/albert/anaconda2/lib/python2.7/site-packages/dask/async.py", line 267, in execute_task
    result = _execute_task(task, data)
  File "/home/albert/anaconda2/lib/python2.7/site-packages/dask/async.py", line 248, in _execute_task
    args2 = [_execute_task(a, cache) for a in args]
  File "/home/albert/anaconda2/lib/python2.7/site-packages/dask/async.py", line 249, in _execute_task
    return func(*args2)
  File "/home/albert/lib/python/oocgcm/filtering/linearfilters.py", line 202, in convolve
    xf = im.convolve(x, self.coefficients, mode=mode)
  File "/home/albert/anaconda2/lib/python2.7/site-packages/scipy/ndimage/filters.py", line 755, in convolve
    origin, True)
  File "/home/albert/anaconda2/lib/python2.7/site-packages/scipy/ndimage/filters.py", line 603, in _correlate_or_convolve
    _nd_image.correlate(input, weights, output, mode, cval, origins)


In [33]:
curls = xr.open_mfdataset(tfiles03,concat_dim='time_counter')['socurloverf'].chunk(chunks={'x': 2500, 'y':2500, 'time_counter': 1})
curl0=curls[0]
win_box2D = curl0.win
win_box2D.set(window_name='lanczos', n=[120, 120], dims=['x', 'y'], fc=0.00833)
%timeit bw = win_box2D.boundary_weights(drop_dims=[])
signal_LS = win_box2D.apply(weights=bw)

1 loop, best of 3: 16min 46s per loop
[#####################################   ] | 93% Completed | 46.0s


MemoryError: 

Traceback
---------
  File "/home/albert/anaconda2/lib/python2.7/site-packages/dask/async.py", line 267, in execute_task
    result = _execute_task(task, data)
  File "/home/albert/anaconda2/lib/python2.7/site-packages/dask/async.py", line 248, in _execute_task
    args2 = [_execute_task(a, cache) for a in args]
  File "/home/albert/anaconda2/lib/python2.7/site-packages/dask/async.py", line 249, in _execute_task
    return func(*args2)
  File "/home/albert/lib/python/oocgcm/filtering/linearfilters.py", line 202, in convolve
    xf = im.convolve(x, self.coefficients, mode=mode)
  File "/home/albert/anaconda2/lib/python2.7/site-packages/scipy/ndimage/filters.py", line 755, in convolve
    origin, True)
  File "/home/albert/anaconda2/lib/python2.7/site-packages/scipy/ndimage/filters.py", line 603, in _correlate_or_convolve
    _nd_image.correlate(input, weights, output, mode, cval, origins)


In [34]:
curls = xr.open_mfdataset(tfiles03,concat_dim='time_counter')['socurloverf'].chunk(chunks={'x': 3000, 'y':3000, 'time_counter': 1})
curl0=curls[0]
win_box2D = curl0.win
win_box2D.set(window_name='lanczos', n=[120, 120], dims=['x', 'y'], fc=0.00833)
%timeit bw = win_box2D.boundary_weights(drop_dims=[])
signal_LS = win_box2D.apply(weights=bw)

1 loop, best of 3: 16min 38s per loop
[#####################################   ] | 94% Completed | 45.2s


MemoryError: 

Traceback
---------
  File "/home/albert/anaconda2/lib/python2.7/site-packages/dask/async.py", line 267, in execute_task
    result = _execute_task(task, data)
  File "/home/albert/anaconda2/lib/python2.7/site-packages/dask/async.py", line 248, in _execute_task
    args2 = [_execute_task(a, cache) for a in args]
  File "/home/albert/anaconda2/lib/python2.7/site-packages/dask/async.py", line 249, in _execute_task
    return func(*args2)
  File "/home/albert/lib/python/oocgcm/filtering/linearfilters.py", line 202, in convolve
    xf = im.convolve(x, self.coefficients, mode=mode)
  File "/home/albert/anaconda2/lib/python2.7/site-packages/scipy/ndimage/filters.py", line 755, in convolve
    origin, True)
  File "/home/albert/anaconda2/lib/python2.7/site-packages/scipy/ndimage/filters.py", line 603, in _correlate_or_convolve
    _nd_image.correlate(input, weights, output, mode, cval, origins)


In [35]:
curls = xr.open_mfdataset(tfiles03,concat_dim='time_counter')['socurloverf'].chunk(chunks={'x': 3500, 'y':3500, 'time_counter': 1})
curl0=curls[0]
win_box2D = curl0.win
win_box2D.set(window_name='lanczos', n=[120, 120], dims=['x', 'y'], fc=0.00833)
%timeit bw = win_box2D.boundary_weights(drop_dims=[])
signal_LS = win_box2D.apply(weights=bw)

1 loop, best of 3: 16min 35s per loop
[#####################################   ] | 94% Completed | 44.2s


MemoryError: 

Traceback
---------
  File "/home/albert/anaconda2/lib/python2.7/site-packages/dask/async.py", line 267, in execute_task
    result = _execute_task(task, data)
  File "/home/albert/anaconda2/lib/python2.7/site-packages/dask/async.py", line 248, in _execute_task
    args2 = [_execute_task(a, cache) for a in args]
  File "/home/albert/anaconda2/lib/python2.7/site-packages/dask/async.py", line 249, in _execute_task
    return func(*args2)
  File "/home/albert/lib/python/oocgcm/filtering/linearfilters.py", line 202, in convolve
    xf = im.convolve(x, self.coefficients, mode=mode)
  File "/home/albert/anaconda2/lib/python2.7/site-packages/scipy/ndimage/filters.py", line 755, in convolve
    origin, True)
  File "/home/albert/anaconda2/lib/python2.7/site-packages/scipy/ndimage/filters.py", line 603, in _correlate_or_convolve
    _nd_image.correlate(input, weights, output, mode, cval, origins)


In [36]:
curls = xr.open_mfdataset(tfiles03,concat_dim='time_counter')['socurloverf'].chunk(chunks={'x': 4000, 'y':4000, 'time_counter': 1})
curl0=curls[0]
win_box2D = curl0.win
win_box2D.set(window_name='lanczos', n=[120, 120], dims=['x', 'y'], fc=0.00833)
%timeit bw = win_box2D.boundary_weights(drop_dims=[])
signal_LS = win_box2D.apply(weights=bw)

1 loop, best of 3: 16min 34s per loop
[#####################################   ] | 94% Completed | 44.4s


MemoryError: 

Traceback
---------
  File "/home/albert/anaconda2/lib/python2.7/site-packages/dask/async.py", line 267, in execute_task
    result = _execute_task(task, data)
  File "/home/albert/anaconda2/lib/python2.7/site-packages/dask/async.py", line 248, in _execute_task
    args2 = [_execute_task(a, cache) for a in args]
  File "/home/albert/anaconda2/lib/python2.7/site-packages/dask/async.py", line 249, in _execute_task
    return func(*args2)
  File "/home/albert/lib/python/oocgcm/filtering/linearfilters.py", line 202, in convolve
    xf = im.convolve(x, self.coefficients, mode=mode)
  File "/home/albert/anaconda2/lib/python2.7/site-packages/scipy/ndimage/filters.py", line 755, in convolve
    origin, True)
  File "/home/albert/anaconda2/lib/python2.7/site-packages/scipy/ndimage/filters.py", line 603, in _correlate_or_convolve
    _nd_image.correlate(input, weights, output, mode, cval, origins)


In [38]:
curls = xr.open_mfdataset(tfiles03,concat_dim='time_counter')['socurloverf']
curl0=curls[0]
win_box2D = curl0.win
win_box2D.set(window_name='lanczos', n=[120, 120], dims=['x', 'y'], fc=0.00833)
bw = win_box2D.boundary_weights(drop_dims=[])
signal_LS = win_box2D.apply(weights=bw)

[########################################] | 100% Completed | 18min 33.3s


In [41]:
win_box2D = curls.win
win_box2D.set(window_name='lanczos', n=[120, 120], dims=['x', 'y'], fc=0.00833)
bw = win_box2D.boundary_weights(drop_dims=[])
signal_LS = win_box2D.apply(weights=bw)

[#######                                 ] | 19% Completed |  1min  7.2s


MemoryError: 

Traceback
---------
  File "/home/albert/anaconda2/lib/python2.7/site-packages/dask/async.py", line 267, in execute_task
    result = _execute_task(task, data)
  File "/home/albert/anaconda2/lib/python2.7/site-packages/dask/async.py", line 248, in _execute_task
    args2 = [_execute_task(a, cache) for a in args]
  File "/home/albert/anaconda2/lib/python2.7/site-packages/dask/async.py", line 249, in _execute_task
    return func(*args2)
  File "/home/albert/lib/python/oocgcm/filtering/linearfilters.py", line 202, in convolve
    xf = im.convolve(x, self.coefficients, mode=mode)
  File "/home/albert/anaconda2/lib/python2.7/site-packages/scipy/ndimage/filters.py", line 755, in convolve
    origin, True)
  File "/home/albert/anaconda2/lib/python2.7/site-packages/scipy/ndimage/filters.py", line 603, in _correlate_or_convolve
    _nd_image.correlate(input, weights, output, mode, cval, origins)


In [44]:
curl1=curls[0:4]
win_box2D = curl1.win
win_box2D.set(window_name='lanczos', n=[120, 120], dims=['x', 'y'], fc=0.00833)
bw = win_box2D.boundary_weights(drop_dims=[])
signal_LS = win_box2D.apply(weights=bw)

[##############################          ] | 76% Completed | 56.8s


MemoryError: 

Traceback
---------
  File "/home/albert/anaconda2/lib/python2.7/site-packages/dask/async.py", line 267, in execute_task
    result = _execute_task(task, data)
  File "/home/albert/anaconda2/lib/python2.7/site-packages/dask/async.py", line 248, in _execute_task
    args2 = [_execute_task(a, cache) for a in args]
  File "/home/albert/anaconda2/lib/python2.7/site-packages/dask/async.py", line 249, in _execute_task
    return func(*args2)
  File "/home/albert/lib/python/oocgcm/filtering/linearfilters.py", line 202, in convolve
    xf = im.convolve(x, self.coefficients, mode=mode)
  File "/home/albert/anaconda2/lib/python2.7/site-packages/scipy/ndimage/filters.py", line 755, in convolve
    origin, True)
  File "/home/albert/anaconda2/lib/python2.7/site-packages/scipy/ndimage/filters.py", line 603, in _correlate_or_convolve
    _nd_image.correlate(input, weights, output, mode, cval, origins)


In [5]:
curls = xr.open_mfdataset(tfiles03,concat_dim='time_counter')['socurloverf'].chunk(chunks={'time_counter': 1})
curl1=curls[0:2]
win_box2D = curl1.win
win_box2D.set(window_name='lanczos', n=[120, 120], dims=['x', 'y'], fc=0.00833)
bw = win_box2D.boundary_weights(drop_dims=[])
signal_LS = win_box2D.apply(weights=bw)

/home/albert/lib/python/oocgcm/filtering/linearfilters.py:58: RuntimeWarning: invalid value encountered in divide
  np.sin(np.pi * k / (n / 2.)) / (np.pi * k / (n / 2.)))


[######################################  ] | 96% Completed | 55.2s


MemoryError: 

Traceback
---------
  File "/home/albert/anaconda2/lib/python2.7/site-packages/dask/async.py", line 267, in execute_task
    result = _execute_task(task, data)
  File "/home/albert/anaconda2/lib/python2.7/site-packages/dask/async.py", line 248, in _execute_task
    args2 = [_execute_task(a, cache) for a in args]
  File "/home/albert/anaconda2/lib/python2.7/site-packages/dask/async.py", line 249, in _execute_task
    return func(*args2)
  File "/home/albert/lib/python/oocgcm/filtering/linearfilters.py", line 202, in convolve
    xf = im.convolve(x, self.coefficients, mode=mode)
  File "/home/albert/anaconda2/lib/python2.7/site-packages/scipy/ndimage/filters.py", line 755, in convolve
    origin, True)
  File "/home/albert/anaconda2/lib/python2.7/site-packages/scipy/ndimage/filters.py", line 603, in _correlate_or_convolve
    _nd_image.correlate(input, weights, output, mode, cval, origins)


In [1]:
import netCDF4

In [13]:
from joblib import Parallel, delayed
import multiprocessing
inputs = range(10) 
def processInput(i):
  return i * i
num_cores = multiprocessing.cpu_count()
results = Parallel(n_jobs=num_cores)(delayed(processInput)(i) for i in inputs)
results

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [15]:
inputs = range(2) 
inputs

[0, 1]

In [ ]:
inputs = range(2) 
def processInput(i):
    curl1=curls[i] 
    win_box2D = curl1.win
    win_box2D.set(window_name='lanczos', n=[120, 120], dims=['x', 'y'], fc=0.00833)
    bw = win_box2D.boundary_weights(drop_dims=[])
    signal_LS = win_box2D.apply(weights=bw)
    return signal_LS

num_cores = multiprocessing.cpu_count()
results = Parallel(n_jobs=num_cores)(delayed(processInput)(i) for i in inputs)
